# Pandas Data Type Mismatch

This notebook explains how to identify data in columns with the wrong data type with `pandas`.

### Packages

This tutorial uses:
* [pandas](https://pandas.pydata.org/docs/)
* [datetime](https://docs.python.org/3/library/datetime.html)

In [1]:
import pandas as pd
import datetime  # datetime module adds functionality to manipulate date and time data
import numpy as np

datetime objects can be categorized into two categories  
1. aware objects : an aware objects contain enough information to locate itself relative to the other aware objects given the knowledge of algorithmic and political time adjustments.
2. naive objects: a naive objects do not contain that kind of information to locate itself relative to other naive objects

## Creating the data

We will create a dataframe that contains multiple occurances of duplication for this example.

In [22]:
df = pd.DataFrame({'A': ['text']*20,  # ['text']*20 creates a list of length 20 with the same element 'text' in it
                   'B': [1, 2.2]*10,  # [1, 2.2]*10 creates a list of length 20 with the repetition of the two original elements 
                   'C': [True, False]*10,  # [True, False]*10 will create a length 20 list with alternating True and False
                   'D': pd.to_datetime('2020-01-01'), # this line will create a pandas timestamp  object and fill all the rows of 'E' with the same object
                #    'E': 5
                  })

In [23]:
df.head()

,A,B,C,D
0,text,1.0,True,2020-01-01
1,text,2.2,False,2020-01-01
2,text,1.0,True,2020-01-01
3,text,2.2,False,2020-01-01
4,text,1.0,True,2020-01-01


Next, add some mistyped data to the dataframe.

In [31]:
df.iloc[0,0] = 1
df.iloc[1,0] = -2
df.iloc[10,0] = pd.to_datetime('2021-01-01')
df.iloc[5,1] = '2.2'
df.iloc[7,1] = 'A+B'
df.iloc[4,2] = 1
df.iloc[5,2] = 'False'
df.iloc[9,2] = -12.6
df.iloc[12,2] = 'text'
df.iloc[2,3] = 12
df.iloc[12,3] = '2020-01-01'
df

,A,B,C,D
0,1,1.0,True,2020-01-01 00:00:00
1,-2,2.2,False,2020-01-01 00:00:00
2,text,1.0,True,12
3,text,2.2,False,2020-01-01 00:00:00
4,text,1.0,1,2020-01-01 00:00:00
5,text,2.2,False,2020-01-01 00:00:00
6,text,1.0,True,2020-01-01 00:00:00
7,text,A+B,False,2020-01-01 00:00:00
8,text,1.0,True,2020-01-01 00:00:00
9,text,2.2,-12.6,2020-01-01 00:00:00


## Identify mistyped data

The function `applymap` and `isinstance` will return a Boolean dataframe with **True** when the data type matches and **False** when the data type does not match.

### Check numeric

In [32]:
numeric = df.applymap(lambda x: isinstance(x, (int, float))) # this line will check every value in the dataframe
                                                             # and return True the datatype is int or float and flase if the datatype is anything else.
numeric

,A,B,C,D
0,True,True,True,False
1,True,True,True,False
2,False,True,True,True
3,False,True,True,False
4,False,True,True,False
5,False,False,False,False
6,False,True,True,False
7,False,False,True,False
8,False,True,True,False
9,False,True,True,False


Since only column **B** is supposed to be numeric, this can be made more specific by running `applymap` only on column **B**.

In [33]:
numeric = df.applymap(lambda x: isinstance(x, (int, float)))['B'] # this will check the column 'B' only
numeric

0      True
1      True
2      True
3      True
4      True
5     False
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: B, dtype: bool

In [43]:
~numeric

0     False
1     False
2     False
3     False
4     False
5      True
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: B, dtype: bool

Using this Boolean series to return the non-numeric data

In [34]:
df[~numeric] # this line will select the data for which the value of numeric is False
             # if we want to select the True we can write df[numeric]

,A,B,C,D
5,text,2.2,False,2020-01-01 00:00:00
7,text,A+B,False,2020-01-01 00:00:00


### Check datetime

In [37]:
dt = df.applymap(lambda x: isinstance(x, (datetime.datetime)))['D']
dt

0      True
1      True
2     False
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12    False
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: D, dtype: bool

Using this Boolean series to return the non-numeric data

In [38]:
df[~dt]

,A,B,C,D
2,text,1.0,True,12
12,text,1.0,text,2020-01-01


### Check string

In [39]:
strings = df.applymap(lambda x: isinstance(x, (str)))['A']
strings

0     False
1     False
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10    False
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: A, dtype: bool

Using this Boolean series to return the non-numeric data

In [40]:
df[~strings]

,A,B,C,D
0,1,1.0,True,2020-01-01 00:00:00
1,-2,2.2,False,2020-01-01 00:00:00
10,2021-01-01 00:00:00,1.0,True,2020-01-01 00:00:00


### Check Boolean

In [41]:
torf = df.applymap(lambda x: isinstance(x, (bool)))['C']
torf

0      True
1      True
2      True
3      True
4     False
5     False
6      True
7      True
8      True
9     False
10     True
11     True
12    False
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: C, dtype: bool

Using this Boolean series to return the non-numeric data

In [42]:
df[~torf]

,A,B,C,D
4,text,1.0,1,2020-01-01 00:00:00
5,text,2.2,False,2020-01-01 00:00:00
9,text,2.2,-12.6,2020-01-01 00:00:00
12,text,1.0,text,2020-01-01
